In [8]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
ds = pd.read_json("datasets/matches_parsed_2018_2020_astralis.json")

In [11]:
# team = "Astralis"
# ds = ds[(ds["team1"] == team) | (ds["team2"] == team) | (ds["team1"] == team) | (ds["team2"] == team)]

In [12]:
# ds = ds[["team1", "team2", "map", "team1win"]]
ds = ds[["team2", "map", "team1win"]]

In [13]:
ds[0:10]

,team2,map,team1win
0,G2,nuke,0
1,G2,d2,0
2,North,inf,1
3,North,vtg,1
4,Heroic,inf,1
5,Heroic,trn,1
6,G2,vtg,1
7,G2,nuke,1
8,FaZe,inf,1
9,FaZe,d2,1


In [14]:
len(ds)

229

In [15]:
X = ds.iloc[:, :-1].values
y = ds.iloc[:, -1].values

### Encode Fatures

In [16]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1])], remainder='passthrough')

In [17]:
encoder = ct.fit(X)

In [18]:
X = encoder.transform(X).toarray()

In [19]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Encode Variable

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
res = list()
for cl in le.classes_:
    res.append({"name": cl, "label": le.transform([cl])[0]})

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [21]:
from sklearn.naive_bayes import GaussianNB

In [22]:
classifier = GaussianNB()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

In [23]:
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=5)
print("Accuracy is \x1b[1;31m{}%\x1b[0m".format(round(accuracies.mean().round(3) * 100,2)))

Accuracy is 45.6%


d:\users\thiago\documents\applications\hltv-crawler\notebooks\.venv\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [27]:
from sklearn.tree import DecisionTreeClassifier

In [28]:
classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

In [29]:
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=5)
print("Accuracy is \x1b[1;31m{}%\x1b[0m".format(round(accuracies.mean().round(3) * 100,2)))

Accuracy is 63.8%


d:\users\thiago\documents\applications\hltv-crawler\notebooks\.venv\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier(n_estimators=25, criterion='gini', random_state=0)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

In [ ]:
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=5)
print("Accuracy is \x1b[1;31m{}%\x1b[0m".format(round(accuracies.mean().round(3) * 100,2)))

### Testing the model

In [ ]:
ds.map.unique()

In [ ]:
ds.team2.unique()

In [ ]:
team1 = 'MIBR'
team2 = 'FURIA'

In [ ]:
map = 'mrg'
pick_team = 'FURIA'
other_team = team2 if pick_team == team1 else team1

print(f"{team1} vs {team2} // Map: {map} // pick: {pick_team}")
print(f"{pick_team} vs {other_team}")

ds_test = [[pick_team, other_team, 'd2']]
X_new = encoder.transform(ds_test).toarray()
y_pred = classifier.predict_proba(X_new)

# winner = pick_team if y_pred[0] == 0 else other_team
# print(f'{winner} Ganha')
probabilities = list(zip(classifier.classes_, classifier.predict_proba(X_new)[0]))
probabilities_translate = list()
for prob in probabilities:
    team = [x["name"] for x in res if x["label"] == prob[0]][0]
    probabilities_translate.append({"team": team, "win_probability": prob[1]})
prob_df = pd.DataFrame(probabilities_translate).sort_values("win_probability", ascending=False)
prob_df[0:2]

In [ ]:
probabilities_translate = list()
for prob in probabilities:
    team = [x["name"] for x in res if x["label"] == prob[0]][0]
    probabilities_translate.append({"team": team, "win_probability": prob[1]})
prob_df = pd.DataFrame(probabilities_translate).sort_values("win_probability", ascending=False)
prob_df[0:2]

In [ ]:
team2 = "NiP"
game_map = "d2"

ds_test = [[team2, game_map]]
X_new = encoder.transform(ds_test).toarray()
y_pred = classifier.predict(X_new)

win = True if y_pred[0] == 1 else False
print("Victory" if win else "Loose")